In [1]:
import pandas as pd

# Load the ratings file
ratings_df = pd.read_csv(
    'ratings.dat',
    sep='::',
    engine='python',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp']
)

# Load the movies file (Crucial: use encoding='latin-1' for special characters)
movies_df = pd.read_csv(
    'movies.dat',
    sep='::',
    engine='python',
    encoding='latin-1',
    names=['MovieID', 'Title', 'Genres']
)

# Load the users file
users_df = pd.read_csv(
    'users.dat',
    sep='::',
    engine='python',
    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
)

print("--- Data Loading Complete ---")
print("Ratings Count:", len(ratings_df))

--- Data Loading Complete ---
Ratings Count: 1000209


In [2]:
# Merge ratings and users on UserID, then merge the result with movies on MovieID
master_df = ratings_df.merge(users_df, on='UserID').merge(movies_df, on='MovieID')

print("--- Data Merging Complete ---")
print("Master DataFrame head (first 5 rows):")
print(master_df.head())
print("\nColumns and Data Types:")
master_df.info()

--- Data Merging Complete ---
Master DataFrame head (first 5 rows):
   UserID  MovieID  Rating  Timestamp Gender  Age  Occupation Zip-code  \
0       1     1193       5  978300760      F    1          10    48067   
1       1      661       3  978302109      F    1          10    48067   
2       1      914       3  978301968      F    1          10    48067   
3       1     3408       4  978300275      F    1          10    48067   
4       1     2355       5  978824291      F    1          10    48067   

                                    Title                        Genres  
0  One Flew Over the Cuckoo's Nest (1975)                         Drama  
1        James and the Giant Peach (1996)  Animation|Children's|Musical  
2                     My Fair Lady (1964)               Musical|Romance  
3                  Erin Brockovich (2000)                         Drama  
4                    Bug's Life, A (1998)   Animation|Children's|Comedy  

Columns and Data Types:
<class 'pandas.cor

In [3]:
# --- 3A. Convert Timestamps (Pandas Task) ---
# Convert the Unix timestamp (seconds) into a readable datetime format
master_df['DateTime'] = pd.to_datetime(master_df['Timestamp'], unit='s')
master_df['Year'] = master_df['DateTime'].dt.year
master_df['Date'] = master_df['DateTime'].dt.date

# --- 3B. Expand Genres (Critical for Genre Analysis) ---
# Create a new DataFrame (df_genre) where one movie rating becomes multiple rows, 
# one for each genre, by splitting the '|' delimiter and exploding the result.
df_genre = master_df.assign(
    Genre=master_df['Genres'].str.split('|')
).explode('Genre')

print("\n--- Cleaning and Expansion Complete ---")
print("Example of Genre Expansion (first 5 rows of df_genre):")
print(df_genre[['Title', 'Genres', 'Genre', 'Rating']].head())


--- Cleaning and Expansion Complete ---
Example of Genre Expansion (first 5 rows of df_genre):
                                    Title                        Genres  \
0  One Flew Over the Cuckoo's Nest (1975)                         Drama   
1        James and the Giant Peach (1996)  Animation|Children's|Musical   
1        James and the Giant Peach (1996)  Animation|Children's|Musical   
1        James and the Giant Peach (1996)  Animation|Children's|Musical   
2                     My Fair Lady (1964)               Musical|Romance   

        Genre  Rating  
0       Drama       5  
1   Animation       3  
1  Children's       3  
1     Musical       3  
2     Musical       3  


In [4]:
# Create a matrix where rows are users, columns are movies, and values are ratings
# Note: This matrix will be very sparse (mostly NaNs), which is normal.
user_movie_pivot = master_df.pivot_table(
    index='UserID',
    columns='Title',
    values='Rating'
)

print("--- User-Movie Pivot Table Created (for Recommendation Prep) ---")
print("Dimensions:", user_movie_pivot.shape)
print("Example (first 5 users and movies):")
print(user_movie_pivot.iloc[:5, :5])

--- User-Movie Pivot Table Created (for Recommendation Prep) ---
Dimensions: (6040, 3706)
Example (first 5 users and movies):
Title   $1,000,000 Duck (1971)  'Night Mother (1986)  \
UserID                                                 
1                          NaN                   NaN   
2                          NaN                   NaN   
3                          NaN                   NaN   
4                          NaN                   NaN   
5                          NaN                   NaN   

Title   'Til There Was You (1997)  'burbs, The (1989)  \
UserID                                                  
1                             NaN                 NaN   
2                             NaN                 NaN   
3                             NaN                 NaN   
4                             NaN                 NaN   
5                             NaN                 NaN   

Title   ...And Justice for All (1979)  
UserID                                 
1

In [5]:
# Group the genre-expanded data by 'Year' and 'Genre' and count the number of ratings
genre_popularity_trend = df_genre.groupby(['Year', 'Genre'])['Rating'].count().reset_index(name='Rating_Count')

# Pivot the data so years are rows and genres are columns for trend analysis
genre_popularity_pivot = genre_popularity_trend.pivot(index='Year', columns='Genre', values='Rating_Count').fillna(0)

print("\n--- Genre Popularity Trends by Year ---")
print("Example (first 5 years of the pivot table):")
print(genre_popularity_pivot.head())


--- Genre Popularity Trends by Year ---
Example (first 5 years of the pivot table):
Genre  Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  \
Year                                                                          
2000   238212     124112      38951       65130  321496  72632         6941   
2001    14021       7055       3301        5228   25136   5010          663   
2002     4553       2436        944        1636    8685   1648          245   
2003      671        350         97         192    1263    251           61   

Genre   Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  \
Year                                                                           
2000   318840    33564      16898   69916    37458    36681   133796  146085   
2001    25226     1885        963    4526     2973     2516     9651    7842   
2002     9210      743        345    1740      965      877     3576    2927   
2003     1253      109         55     20

In [9]:
import warnings

# --- E. Recommend top 5 movies per genre (Pandas Task) ---
print("\n--- Top 5 Recommended Movies per Genre (Content-Based) ---")
genre_movie_stats = df_genre.groupby(['Genre', 'Title']).agg(
    Avg_Rating=('Rating', 'mean'),
    Rating_Count=('Rating', 'count')
).reset_index()
min_ratings = 100
valid_genre_movies = genre_movie_stats[genre_movie_stats['Rating_Count'] >= min_ratings]
def get_top_n(group, n=5):
    return group.sort_values(by='Avg_Rating', ascending=False).head(n)

# Suppress the Pandas Deprecation Warning temporarily
with warnings.catch_warnings():
    warnings.simplefilter("ignore") 
    
    # Apply the function with the .reset_index() fix
    top_5_per_genre = valid_genre_movies.groupby('Genre', group_keys=False).apply(get_top_n).reset_index()

print("Top 5 Action Movies:")
# The .reset_index() fix ensures we can use this standard column filter:
print(top_5_per_genre[top_5_per_genre['Genre'] == 'Action'])


--- Top 5 Recommended Movies per Genre (Content-Based) ---
Top 5 Action Movies:
   index   Genre                                              Title  \
0    373  Action  Seven Samurai (The Magnificent Seven) (Shichin...   
1    169  Action                              Godfather, The (1972)   
2    339  Action                     Raiders of the Lost Ark (1981)   
3    409  Action          Star Wars: Episode IV - A New Hope (1977)   
4    170  Action                     Godfather: Part II, The (1974)   

   Avg_Rating  Rating_Count  
0    4.560510           628  
1    4.524966          2223  
2    4.477725          2514  
3    4.453694          2991  
4    4.357565          1692  


In [11]:
import sqlite3
import pandas as pd

# This line creates the connection object 'conn' to an in-memory database
conn = sqlite3.connect(':memory:')

# This loads your prepared Pandas DataFrames into SQL tables within 'conn'
master_df.to_sql('RatingsData', conn, if_exists='replace', index=False)
df_genre.to_sql('GenreData', conn, if_exists='replace', index=False)

print(" SQL Database Connection ('conn') established. You can now run the queries.")

 SQL Database Connection ('conn') established. You can now run the queries.


In [13]:
sql_query_top_10 = """
SELECT 
    Title,
    CAST(AVG(Rating) AS REAL) AS AvgRating,
    COUNT(Rating) AS RatingCount
FROM 
    RatingsData
GROUP BY 
    Title
HAVING 
    COUNT(Rating) >= 100
ORDER BY 
    AvgRating DESC, RatingCount DESC
LIMIT 10;
"""

top_10_movies_sql = pd.read_sql(sql_query_top_10, conn)

print("\n--- SQL Task: Top 10 Highest-Rated Movies (Min 100 Ratings) ---")
print(top_10_movies_sql)


--- SQL Task: Top 10 Highest-Rated Movies (Min 100 Ratings) ---
                                               Title  AvgRating  RatingCount
0  Seven Samurai (The Magnificent Seven) (Shichin...   4.560510          628
1                   Shawshank Redemption, The (1994)   4.554558         2227
2                              Godfather, The (1972)   4.524966         2223
3                              Close Shave, A (1995)   4.520548          657
4                         Usual Suspects, The (1995)   4.517106         1783
5                            Schindler's List (1993)   4.510417         2304
6                         Wrong Trousers, The (1993)   4.507937          882
7      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   4.491489          470
8                     Raiders of the Lost Ark (1981)   4.477725         2514
9                                 Rear Window (1954)   4.476190         1050


In [14]:
sql_query_active_users = """
SELECT 
    UserID,
    Gender,
    Age,
    COUNT(Rating) AS RatingCount
FROM 
    RatingsData
GROUP BY 
    UserID
ORDER BY 
    RatingCount DESC
LIMIT 10;
"""

most_active_users = pd.read_sql(sql_query_active_users, conn)

print("\n--- SQL Task: Top 10 Most Active Users ---")
print(most_active_users)


--- SQL Task: Top 10 Most Active Users ---
   UserID Gender  Age  RatingCount
0    4169      M   50         2314
1    1680      M   25         1850
2    4277      M   35         1743
3    1941      M   35         1595
4    1181      M   35         1521
5     889      M   45         1518
6    3618      M   56         1344
7    2063      M   25         1323
8    1150      F   25         1302
9    1015      M   35         1286


In [15]:
sql_query_avg_rating = """
SELECT 
    Title,
    CAST(AVG(Rating) AS REAL) AS AvgRating,
    COUNT(Rating) AS RatingCount
FROM 
    RatingsData
GROUP BY 
    Title
ORDER BY 
    AvgRating DESC;
"""

avg_rating_per_movie = pd.read_sql(sql_query_avg_rating, conn)

print("\n--- SQL Task: Average Rating Per Movie (Top 5 Overall) ---")
print(avg_rating_per_movie.head())


--- SQL Task: Average Rating Per Movie (Top 5 Overall) ---
                                       Title  AvgRating  RatingCount
0                    Ulysses (Ulisse) (1954)        5.0            1
1                     Song of Freedom (1936)        5.0            1
2                       Smashing Time (1967)        5.0            2
3  Schlafes Bruder (Brother of Sleep) (1995)        5.0            1
4                   One Little Indian (1973)        5.0            1


In [16]:
sql_query_genre_combo = """
SELECT 
    Genres,
    COUNT(Genres) AS CombinationCount
FROM 
    RatingsData
GROUP BY 
    Genres
ORDER BY 
    CombinationCount DESC
LIMIT 10;
"""

common_genre_combos = pd.read_sql(sql_query_genre_combo, conn)

print("\n--- SQL Task: Top 10 Most Common Genre Combinations ---")
print(common_genre_combos)


--- SQL Task: Top 10 Most Common Genre Combinations ---
                    Genres  CombinationCount
0                   Comedy            116883
1                    Drama            111423
2           Comedy|Romance             42712
3             Comedy|Drama             42245
4            Drama|Romance             29170
5          Action|Thriller             26759
6                   Horror             22563
7           Drama|Thriller             18248
8                 Thriller             17851
9  Action|Adventure|Sci-Fi             17783


In [17]:
sql_query_high_rated_last_year = """
SELECT 
    Title,
    AVG(Rating) AS AvgRating,
    COUNT(Rating) AS RatingCount
FROM 
    RatingsData
WHERE 
    Year = 2000 
GROUP BY 
    Title
HAVING 
    AVG(Rating) >= 4.5 AND COUNT(Rating) >= 10
ORDER BY 
    AvgRating DESC
LIMIT 10;
"""

high_rated_last_year = pd.read_sql(sql_query_high_rated_last_year, conn)

print("\n--- SQL Task: Top Movies Rated > 4.5 in the Last Year (2000) ---")
print(high_rated_last_year)


--- SQL Task: Top Movies Rated > 4.5 in the Last Year (2000) ---
                                               Title  AvgRating  RatingCount
0                                     Sanjuro (1962)   4.610169           59
1  Seven Samurai (The Magnificent Seven) (Shichin...   4.572680          571
2                   Shawshank Redemption, The (1994)   4.558908         2088
3                              Close Shave, A (1995)   4.529984          617
4                         Usual Suspects, The (1995)   4.525899         1641
5                         Wrong Trousers, The (1993)   4.522479          823
6                              Godfather, The (1972)   4.514990         2068
7                            Schindler's List (1993)   4.508924         2185
8      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   4.500000          428


In [18]:
# Save the genre-expanded DataFrame, which is the most comprehensive, to a CSV file
df_genre.to_csv('MovieLens_Final_DA_Data.csv', index=False, encoding='utf-8')

print("\n--- Data Export Complete ---")
print("File 'MovieLens_Final_DA_Data.csv' saved to your project folder.")
print("You can now import this file into Power BI to build your dashboard!")


--- Data Export Complete ---
File 'MovieLens_Final_DA_Data.csv' saved to your project folder.
You can now import this file into Power BI to build your dashboard!
